In [ ]:
import geopandas as gpd
from ipyfilechooser import FileChooser
import numpy as np
from pathlib import Path
import rioxarray as rxr
import rioxarray.merge
from tqdm.auto import tqdm
from typing import List

import s3fs
from shapely.geometry import mapping

### Select the chip shape file

In [ ]:
print("Select the chip shape file generated in Notebook 0")
fc = FileChooser(Path.cwd())
display(fc)

### Load the chip geometry from the chip shapefile

In [ ]:
shape_path = Path(fc.selected)
shape_path

In [ ]:
geometry_gdf = gpd.read_file(shape_path)

### Collect the chip directory and ID

In [ ]:
chip_dir = shape_path.parent
chip_id = chip_dir.name.split('_')[-1]
chip_dir

### Load Pekel Tile geojson into a dataframe

In [ ]:
peckel_tile_gdf = gpd.read_file('peckel_tiles.geojson')
peckel_tile_gdf.head()

### Create a dataframe of tiles intersecting the chip

In [ ]:
intersections = peckel_tile_gdf.geometry.intersects(geometry)
peckel_subset = peckel_tile_gdf[intersections].reset_index(drop=True)
peckel_subset

### Download tiles and clip them to chip geometry

In [ ]:
to_merge = list()
for url in peckel_subset.source_url:
    print(url)
    tile = rxr.open_rasterio(url, masked=True).squeeze()
    tile = tile.rio.clip(geometry_gdf.geometry.apply(mapping), 'EPSG:4326')
    to_merge.append(tile)
to_merge

### Merge clipped tiles and output to GeoTiff

In [ ]:
merged = rxr.merge.merge_arrays(to_merge)
merged.rio.to_raster(chip_dir/f"pekel_occurance_{chip_id}.tif")